# <center> Статистический анализ свойств инструмента


Входные данные: файл с котировками, формат CSV, разделитель - запятая, выгруженный из Trading view.
Формат даты: время в формате ISO

Укажите путь к файлу, относительный или абсолютный:

In [19]:
# Пример: 'data/BITTREX_DOGEUSD, 60 (2).csv', 'E:\Projects\data\BITTREX_DOGEUSD, 60 (2).csv'

path_csv = 'E:\Математика финансов\Котировки\BINANCE_ETHUSD, 1D.csv'

In [20]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go

from plotly.subplots import make_subplots

In [21]:
data = pd.read_csv(path_csv)

# Создание столбцов для дальнейшей работы и проверка структуры данных
data['HL2'] = (data['high']+data['low'])/2
data['HL2'] = data['HL2'].astype('float32')

data['dt1'] = (data['HL2']/data['HL2'].shift())
data['dt'] = data['dt1'].apply(lambda x: np.log(x))

def dt_sign(x):
    if x<0:
        return 'Падение'
    else:
        return 'Рост'

data['candle color'] = data['close'] - data['open']    
data['dt знак'] = data['candle color'].apply(dt_sign)

data = data.drop(['dt1', 'candle color'], axis=1)

print(data.info())
print(data.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413 entries, 0 to 412
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   time       413 non-null    object 
 1   open       413 non-null    float64
 2   high       413 non-null    float64
 3   low        413 non-null    float64
 4   close      413 non-null    float64
 5   Volume     413 non-null    float64
 6   Volume MA  413 non-null    float64
 7   HL2        413 non-null    float32
 8   dt         412 non-null    float64
 9   dt знак    413 non-null    object 
dtypes: float32(1), float64(7), object(2)
memory usage: 30.8+ KB
None
                        time        open        high         low       close  \
0  2020-11-10T03:00:00+03:00  444.780967  457.807765  435.237420  450.913793   
1  2020-11-11T03:00:00+03:00  450.958698  483.056625  449.316753  463.819343   
2  2020-11-12T03:00:00+03:00  463.680886  484.159298  437.025559  462.799846   
3  2020-11-13T03:00:00+03

In [22]:
# Работа с датами
data['time'] = pd.to_datetime(data['time'])
data['time'] = data['time'].dt.date

#print(data.tail(100))

In [23]:
# Вывод распределения dt
fig = make_subplots(rows=1, cols=2)

# Дисперсия и стандартное отклонение
std = data['dt'].std()
print('Стандартное отклонение dt(%):', round(std*100, 2))
varr = std*std
print('Дисперсия dt:', round(varr, 5))

filtered = data[(data['dt']< std) & (data['dt']>-1*std)]

trace1 = go.Histogram(x=data['dt'], name='Все данные', xbins=dict(start=-1, end=1, size=0.02))

# Рассмотрим подробнее интервал 1 стандартное отклонение
trace2 = go.Histogram(x=filtered['dt'], name='В пределах стандартного отклонения', xbins=dict(start=-1*std, end=1*std, size=std/20))

fig.update_layout(height = 500, width = 1200, title = 'Распределение приращений логарифмов цен')
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.write_image('data/dt.png')
#fig.show()


Стандартное отклонение dt(%): 4.38
Дисперсия dt: 0.00192


![](data/dt.png)

In [24]:
# Вывод графика цены и объёмов
fig = make_subplots(rows=1, cols=2)

trace1 = go.Scatter(x=data['time'], y=data['HL2'], name='Цена (H+L)/2', line_shape='linear')
trace2 = go.Scatter(x=data['time'], y=data['Volume'], name='Объём', line_shape='linear')

fig.update_layout(height = 500, width = 1200, title = 'Объёмы и цены')
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.write_image('data/vol_price.png')
#fig.show()

# Проверка объёмов по дням падения и роста
print('Сравнение объёмов по дням падения и роста:')

vol1 = data[data['dt знак']=='Рост']['Volume'].sum()
vol2 = data[data['dt знак']=='Падение']['Volume'].sum()

print('Рост: ', round(vol1, 2))
print('Падение: ', round(vol2, 2))


Сравнение объёмов по дням падения и роста:
Рост:  63344571.31
Падение:  44395221.1


![](data/vol_price.png)

In [25]:
# Построение АКФ

corr = [] # коэфициенты корреляции
dt_list = ['dt-1', 'dt-2', 'dt-3', 'dt-4', 'dt-5', 'dt-6', 'dt-7', 'dt-8', 'dt-9']

# Расчёт dt-1 ... dt-9
data['dt-1'] = data['dt'].shift() # значение того же столбца предыдущей строки
for i in range (1, 9):
    name = dt_list[i]
    prev_name = dt_list[i-1]
    data[name] = data[prev_name].shift()

# Корреляция с dt-1... dt-9
for i in range (0, 9):
    name = dt_list[i]
    x = data['dt'][10:].corr(data[name][10:]) # начиная с 10 строки, все значения dt... dt-9 непустые
    corr.append(x)

df_corr = pd.DataFrame({'dt': dt_list, 'corr': corr})
print(df_corr)

# Построение графика АКФ и вывод
fig = px.bar(
    x=dt_list,
    y=corr,
    height=500, #высота
    width=700, #ширина
    title='АКФ (x = dt, y = correlation)')
fig.write_image('data/corr.png')
#fig.show()

     dt      corr
0  dt-1  0.234819
1  dt-2 -0.012364
2  dt-3 -0.019932
3  dt-4  0.015354
4  dt-5 -0.038855
5  dt-6  0.111421
6  dt-7 -0.005811
7  dt-8 -0.100987
8  dt-9 -0.009405


![](data/corr.png)

In [26]:
# Удаление служебных столбцов начиная с dt-2
for i in range(1, 9):
    name = dt_list[i]
    data = data.drop(name, axis=1)
    
#print(data.info())

In [27]:
# Расчёт статистического преимущества и средней длины тренда
data['sign'] = (data['dt'] * data['dt-1'] > 0)
data['sign'] = data['sign'].apply(lambda x: 0 if not x else 1)

# Найдём все тренды (приращения одного знака, идущие подряд)
x = list(data['sign'])
trends = [] # список всех длин трендов
trend = 1 # длина текущего тренда
for i in range(len(x)):
    if x[i] == 1:   # если встречаем 1, увеличиваем длину тренда на 1
        trend += 1
    if x[i] == 0:   # если 0, то начинаем отсчёт сначала, а длину текущего тренда сохраняем
        trends.append(trend)
        trend = 1
trends.append(trend)                

# Средняя длина тренда
# С массивом работать удобнее, есть статистические функции
arr_trends = np.array(trends)
trend_mean = round(arr_trends.mean(), 1)

# Количество трендов разной длины
from collections import Counter
count = Counter(trends)

# Вывод графика распределения
trace1 = go.Histogram(y=trends, name='Все тренды', xbins=dict(start=0, end=arr_trends.max(), size=1))

# Линия y = средняя длина
x_list = []
y_list = []
for i in range (0, count[1]):
    x_list.append(i)
    y_list.append(trend_mean)
  
trace2 = go.Scatter(x=x_list, y=y_list, name='Средняя длина', line_shape='linear')

# Объединение двух графиков в один
fig = go.Figure()
fig.update_layout(height=500, width=800, title='Распределение длины трендов (последовательностей приращений одного знака)')
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.write_image('data/trends.png')
#fig.show()

# Статистическое преимущество
trending = data['sign'].mean()
print('Количество ситуаций, когда знаки соседних приращений равны (%): ', round(trending*100, 2))

print('Средняя длина тренда: ', trend_mean, " приращений")

# Процентное соотношение трендов длины 1-3
for i in range (1, 4):
    n = count[i]
    print('Трендов длины ', i, ': ', round(count[i]/(len(trends))*100, 2), '%')


Количество ситуаций, когда знаки соседних приращений равны (%):  57.38
Средняя длина тренда:  2.3  приращений
Трендов длины  1 :  38.98 %
Трендов длины  2 :  26.55 %
Трендов длины  3 :  15.25 %


![](data/trends.png)

In [28]:
# Анализ положительных и отрицательных приращений

# Запишем станд отклонение в базу со знаком, соответствующим знаку приращения, для вывода горизонтальной линии
data['std'] = data['dt'].apply(lambda x: std if x>0 else -1*std)

dt_positive = data[data['dt'] > 0]
dt_negative = data[data['dt'] < 0]

# Положительные приращения
trace1 = go.Scatter(x=dt_positive['time'], y=dt_positive['dt'], mode='markers', name='Приращения')

# Линия y = стандартное отклонение
trace2 = go.Scatter(x=dt_positive['time'], y=dt_positive['std'], name='Станд отклонение', line_shape='linear')

# Объединение двух графиков в один
fig = go.Figure()
fig.update_layout(height=500, width=1000, title='Анализ положительных приращений')
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.write_image('data/dt_pos.png')
#fig.show()


# Отрицательные приращения
trace1 = go.Scatter(x=dt_negative['time'], y=dt_negative['dt'], mode='markers', name='Приращения')

# Линия y = стандартное отклонение
trace2 = go.Scatter(x=dt_negative['time'], y=dt_negative['std'], name='Станд отклонение', line_shape='linear')

# Объединение двух графиков в один
fig = go.Figure()
fig.update_layout(height=500, width=1000, title='Анализ отрицательных приращений')
fig.add_trace(trace1)
fig.add_trace(trace2)
fig.write_image('data/dt_neg.png')
#fig.show()

![](data/dt_pos.png)

![](data/dt_neg.png)

In [29]:
# Анализ приращений в сравнении со стандартным отклонением

# Положительных...
print('Положительные приращения:')
print('Всего: ', dt_positive.shape[0], round(dt_positive.shape[0]/data.shape[0]*100, 2), '%')
mask = (dt_positive['dt'] < std)
n = dt_positive[mask].shape[0] / dt_positive.shape[0]
print('< 1 стандартного отклонения: ', dt_positive[mask].shape[0], round(n*100, 2), '%')

mask = (dt_positive['dt'] < 2*std)
n = dt_positive[mask].shape[0] / dt_positive.shape[0]
print('< 2 стандартных отклонений: ', dt_positive[mask].shape[0], round(n*100, 2), '%')

mask = (dt_positive['dt'] < 3*std)
n = dt_positive[mask].shape[0] / dt_positive.shape[0]
print('< 3 стандартных отклонений: ', dt_positive[mask].shape[0], round(n*100, 2), '%')

# ... и отрицательных
print('Отрицательные приращения:')
print('Всего: ', dt_negative.shape[0], round(dt_negative.shape[0]/data.shape[0]*100, 2), '%')

mask = (dt_negative['dt'] > -1*std)
n = dt_negative[mask].shape[0] / dt_negative.shape[0]
print('< 1 стандартного отклонения: ', dt_negative[mask].shape[0], round(n*100, 2), '%')

mask = (dt_negative['dt'] > -2*std)
n = dt_negative[mask].shape[0] / dt_negative.shape[0]
print('< 2 стандартных отклонений: ', dt_negative[mask].shape[0], round(n*100, 2), '%')

mask = (dt_negative['dt'] > -3*std)
n = dt_negative[mask].shape[0] / dt_negative.shape[0]
print('< 3 стандартных отклонений: ', dt_negative[mask].shape[0], round(n*100, 2), '%')


Положительные приращения:
Всего:  239 57.87 %
< 1 стандартного отклонения:  172 71.97 %
< 2 стандартных отклонений:  232 97.07 %
< 3 стандартных отклонений:  237 99.16 %
Отрицательные приращения:
Всего:  173 41.89 %
< 1 стандартного отклонения:  132 76.3 %
< 2 стандартных отклонений:  161 93.06 %
< 3 стандартных отклонений:  171 98.84 %
